# INF 721 : Projet Kit Start Data

##### Auteur : Hugo Michel MS BGD

## Analyse de données du vendée globe 

### Traitement du tableau "classement final" de la course

Nettoyage du tableau excel contenant le classement final de course du vendée globe

In [1]:
import os
import pandas as pd
import numpy as np
import re
from datetime import datetime

import hashlib
import glob
import matplotlib.pyplot as plt

import tempfile
from zipfile import ZipFile
import shutil
import os
from fnmatch import fnmatch
import re
import math

!pip install Unidecode
import unidecode

import geopandas as gpd # plot datapoints on a world map

In [2]:
# Fonction qui extrait les chiffres contenu dans une chaine de caractères
def extract_numbers(text):
    num = " ".join(re.findall(r"\d+", text))
    return num
    
# Fonction qui extrait les caractères alphanumérique uniquement contenu dans une chaine de caractères
def extract_characters(text):
    sentence = " ".join(re.findall("[a-zA-Z]+", text))
    return sentence

In [3]:
def convert_race_time_seconds(data_time):
    data_string1 = data_time.split("\n")[0].replace("j", "").replace("h", "").replace("min", "").replace("s", "")
    data_string2 = data_string1.split(" ")
    data_string2 = [int(x) for x in data_string2]

    days = data_string2[0] * 3600 * 24
    hours = data_string2[1] * 3600
    minutes = data_string2[2] * 60
    seconds = data_string2[3]

    final_seconds = days + hours + minutes + seconds
    return int(final_seconds)

race_time = convert_race_time_seconds("80j 03h 44min 46s\n-10h 15min 00s")
race_time

6925486

In [4]:
def convert_Ecart_time_seconds(data_time):
    
    if isinstance(data_time, str):
        
        if len(data_time) > 15:
    
            data_string1 = data_time.replace("j", "").replace("h", "").replace("min", "").replace("s", "")
            data_string2 = data_string1.split(" ")
            data_string2 = [int(x) for x in data_string2]

            days = data_string2[0] * 3600 * 24
            hours = data_string2[1] * 3600
            minutes = data_string2[2] * 60
            seconds = data_string2[3]
            
            final_seconds = days + hours + minutes + seconds
        
        if len(data_time) < 15 and len(data_time) > 9:
            
            data_string1 = data_time.replace("h", "").replace("min", "").replace("s", "")
            data_string2 = data_string1.split(" ")
            data_string2 = [int(x) for x in data_string2]

            hours = data_string2[0] * 3600
            minutes = data_string2[1] * 60
            seconds = data_string2[2]
            
            final_seconds = hours + minutes + seconds
        
            
        if len(data_time) == 9:
    
            data_string1 = data_time.replace("min", "").replace("s", "")
            data_string2 = data_string1.split(" ")
            data_string2 = [int(x) for x in data_string2]

            minutes = data_string2[0] * 60
            seconds = data_string2[1]
            
            final_seconds = minutes + seconds
    
    else:
        final_seconds = 0
    
    return int(final_seconds)

race_time = convert_Ecart_time_seconds("16min 35s")
race_time

995

In [5]:
def convert_date_arrivee(data_arrivee):

    data_arrivee1 = data_arrivee.replace("FR", "").strip()
    data_arrivee2 = datetime.strptime(data_arrivee1, '%d/%m/%Y %H:%M:%S')

    return data_arrivee2

test_date_arrivee = convert_date_arrivee("28/01/2021 04:19:46 FR")
print(test_date_arrivee)

2021-01-28 04:19:46


#### Importation des données du dernier classement

In [6]:
data = pd.read_excel("20210305_080000.xlsx", skiprows=3)
data_cleaned = data.copy()

In [7]:
data_renamed_column = data_cleaned.rename({"Rang\nRank": "Last_Rang", 
                                  "Nat. / Voile\nNat. / Sail": "Nat_Voile", 
                                  "Skipper / Bateau\nSkipper / crew": "Skipper_Bateau",
                                 "Unnamed: 7": "Date_Arrivee", 
                                  "Temps de course\nRace time": "Temps_Course_s",
                                 "Unnamed: 13": "Ecart_Premier_s",
                                 "Unnamed: 15": "Ecart_Precedent_s",
                                 "Sur l\'ortho\nOver ortho": "Ortho_Vitesse_kts",
                                 "Unnamed: 17": "Ortho_Distance_nm",
                                 "►": "Rapport_Ortho_Fond_%",
                                 "Sur le fond\nOver ground": "Fond_Vitesse_kts",
                                 "Unnamed: 20": "Fond_Distance_nm"}, axis='columns')
data_renamed_column

,Unnamed: 0,Last_Rang,Nat_Voile,Skipper_Bateau,Date d'arrivée\nArrival date,Unnamed: 5,Unnamed: 6,Date_Arrivee,Temps_Course_s,Unnamed: 9,...,Unnamed: 11,Écarts\nGaps,Ecart_Premier_s,Unnamed: 14,Ecart_Precedent_s,Ortho_Vitesse_kts,Ortho_Distance_nm,Rapport_Ortho_Fond_%,Fond_Vitesse_kts,Fond_Distance_nm
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Au premier\nTo first,NaN,Au précédent\nTo previous,NaN,Vitesse\nSpeed,Distance\nDistance,NaN,Vitesse\nSpeed,Distance\nDistance
1,NaN,1\nARV,\nFRA 17,Yannick Bestaven\nMaître Coq IV,NaN,NaN,NaN,28/01/2021 04:19:46 FR,80j 03h 44min 46s\n-10h 15min 00s,NaN,...,NaN,NaN,NaN,NaN,NaN,12.6 kts,24365.7 nm,117.3 %,14.8 kts,28583.8 nm
2,NaN,2\nARV,\nFRA 79,Charlie Dalin\nAPIVIA,NaN,NaN,NaN,27/01/2021 20:35:47 FR,80j 06h 15min 47s\n,NaN,...,NaN,NaN,02h 31min 01s,NaN,02h 31min 01s,12.6 kts,24365.7 nm,119.6 %,15.1 kts,29135.0 nm
3,NaN,3\nARV,\nFRA 18,Louis Burton\nBureau Vallée 2,NaN,NaN,NaN,28/01/2021 00:45:12 FR,80j 10h 25min 12s\n,NaN,...,NaN,NaN,06h 40min 26s,NaN,04h 09min 25s,12.6 kts,24365.7 nm,117.6 %,14.8 kts,28650.0 nm
4,NaN,4\nARV,\nFRA 01,Jean Le Cam\nYes we Cam !,NaN,NaN,NaN,28/01/2021 20:19:55 FR,80j 13h 44min 55s\n-16h 15min 00s,NaN,...,NaN,NaN,10h 00min 09s,NaN,03h 19min 43s,12.5 kts,24365.7 nm,112.9 %,14.1 kts,27501.5 nm
5,NaN,5\nARV,\nMON 10,Boris Herrmann\nSeaexplorer - Yacht Club De Mo...,NaN,NaN,NaN,28/01/2021 11:19:45 FR,80j 14h 59min 45s\n-06h 00min 00s,NaN,...,NaN,NaN,11h 14min 59s,NaN,01h 14min 50s,12.6 kts,24365.7 nm,116.8 %,14.7 kts,28448.5 nm
6,NaN,6\nARV,\nFRA 59,Thomas Ruyant\nLinkedOut,NaN,NaN,NaN,28/01/2021 05:42:01 FR,80j 15h 22min 01s\n,NaN,...,NaN,NaN,11h 37min 15s,NaN,22min 16s,12.6 kts,24365.7 nm,119.7 %,15.1 kts,29175.5 nm
7,NaN,7\nARV,\nFRA 1000,Damien Seguin\nGroupe APICIL,NaN,NaN,NaN,28/01/2021 12:18:20 FR,80j 21h 58min 20s\n,NaN,...,NaN,NaN,18h 13min 34s,NaN,06h 36min 19s,12.5 kts,24365.7 nm,112.9 %,14.2 kts,27512.3 nm
8,NaN,8\nARV,\nITA 34,Giancarlo Pedote\nPrysmian Group,NaN,NaN,NaN,28/01/2021 13:02:20 FR,80j 22h 42min 20s\n,NaN,...,NaN,NaN,18h 57min 34s,NaN,44min 00s,12.5 kts,24365.7 nm,116.9 %,14.7 kts,28489.9 nm
9,NaN,9\nARV,\nFRA 09,Benjamin Dutreux\nOMIA - Water Family,NaN,NaN,NaN,29/01/2021 10:05:20 FR,81j 19h 45min 20s\n,NaN,...,NaN,NaN,1j 16h 00min 34s,NaN,21h 03min 00s,12.4 kts,24365.7 nm,114.2 %,14.2 kts,27832.5 nm


## Nettoyage du DataFrame

In [8]:
# On supprime la 1ère ligne du tableaux contenant les anciens noms de colonnes
data_renamed_column.drop(index=data_renamed_column.index[0], axis=0, inplace=True)

data_cleaned = data_renamed_column.copy()

# Suprression de la colonne "Unamed: 0"
data_cleaned.pop("Unnamed: 0")
data_cleaned.pop("Date d'arrivée\nArrival date")
data_cleaned.pop("Unnamed: 5")
data_cleaned.pop("Unnamed: 6")
data_cleaned.pop("Unnamed: 9")
data_cleaned.pop("Unnamed: 10")
data_cleaned.pop("Unnamed: 11")
data_cleaned.pop("Écarts\nGaps")
data_cleaned.pop("Unnamed: 14")

# Suppression des 4 dernières lignes d'informations inutiles contenu dans le dataframe
data_cleaned.drop(data_cleaned.index[-4:], inplace=True)
data_cleaned


,Last_Rang,Nat_Voile,Skipper_Bateau,Date_Arrivee,Temps_Course_s,Ecart_Premier_s,Ecart_Precedent_s,Ortho_Vitesse_kts,Ortho_Distance_nm,Rapport_Ortho_Fond_%,Fond_Vitesse_kts,Fond_Distance_nm
1,1\nARV,\nFRA 17,Yannick Bestaven\nMaître Coq IV,28/01/2021 04:19:46 FR,80j 03h 44min 46s\n-10h 15min 00s,NaN,NaN,12.6 kts,24365.7 nm,117.3 %,14.8 kts,28583.8 nm
2,2\nARV,\nFRA 79,Charlie Dalin\nAPIVIA,27/01/2021 20:35:47 FR,80j 06h 15min 47s\n,02h 31min 01s,02h 31min 01s,12.6 kts,24365.7 nm,119.6 %,15.1 kts,29135.0 nm
3,3\nARV,\nFRA 18,Louis Burton\nBureau Vallée 2,28/01/2021 00:45:12 FR,80j 10h 25min 12s\n,06h 40min 26s,04h 09min 25s,12.6 kts,24365.7 nm,117.6 %,14.8 kts,28650.0 nm
4,4\nARV,\nFRA 01,Jean Le Cam\nYes we Cam !,28/01/2021 20:19:55 FR,80j 13h 44min 55s\n-16h 15min 00s,10h 00min 09s,03h 19min 43s,12.5 kts,24365.7 nm,112.9 %,14.1 kts,27501.5 nm
5,5\nARV,\nMON 10,Boris Herrmann\nSeaexplorer - Yacht Club De Mo...,28/01/2021 11:19:45 FR,80j 14h 59min 45s\n-06h 00min 00s,11h 14min 59s,01h 14min 50s,12.6 kts,24365.7 nm,116.8 %,14.7 kts,28448.5 nm
6,6\nARV,\nFRA 59,Thomas Ruyant\nLinkedOut,28/01/2021 05:42:01 FR,80j 15h 22min 01s\n,11h 37min 15s,22min 16s,12.6 kts,24365.7 nm,119.7 %,15.1 kts,29175.5 nm
7,7\nARV,\nFRA 1000,Damien Seguin\nGroupe APICIL,28/01/2021 12:18:20 FR,80j 21h 58min 20s\n,18h 13min 34s,06h 36min 19s,12.5 kts,24365.7 nm,112.9 %,14.2 kts,27512.3 nm
8,8\nARV,\nITA 34,Giancarlo Pedote\nPrysmian Group,28/01/2021 13:02:20 FR,80j 22h 42min 20s\n,18h 57min 34s,44min 00s,12.5 kts,24365.7 nm,116.9 %,14.7 kts,28489.9 nm
9,9\nARV,\nFRA 09,Benjamin Dutreux\nOMIA - Water Family,29/01/2021 10:05:20 FR,81j 19h 45min 20s\n,1j 16h 00min 34s,21h 03min 00s,12.4 kts,24365.7 nm,114.2 %,14.2 kts,27832.5 nm
10,10\nARV,\nFRA 53,Maxime Sorel\nV And B Mayenne,30/01/2021 04:50:15 FR,82j 14h 30min 15s\n,2j 10h 45min 29s,18h 44min 55s,12.3 kts,24365.7 nm,112.2 %,13.8 kts,27346.9 nm


In [9]:

    
# On cherche l'indice à partir du quel le premier skippeur à abandonné
index = data_cleaned.index
condition = data_cleaned["Last_Rang"] == "RET"
rank_RET_indices = index[condition]
rank_RET_indices_list = rank_RET_indices.tolist()
if not rank_RET_indices_list:
    rank_RET = data_cleaned.shape[0] + 1
else:
    rank_RET = rank_RET_indices_list[0]
    
print(rank_RET)

26


In [10]:
# Remove "\n" des colonnes "Nat_Voile", "Skipper_Bateau", "Heure_Enregistrement_Classement"
data_cleaned_bis = data_cleaned

# Nat_Voile
data_cleaned_bis["Nat_Voile"] = data_cleaned_bis["Nat_Voile"].astype(str)
data_cleaned_bis["Nat_Voile"] = data_cleaned_bis["Nat_Voile"].apply(lambda x : x.replace("\n",""))

# Skipper_Bateau
data_cleaned_bis["Skipper_Bateau"] = data_cleaned_bis["Skipper_Bateau"].astype(str)
data_cleaned_bis["Skipper_Bateau"] = data_cleaned_bis["Skipper_Bateau"].apply(lambda x : x.replace("\n"," "))

# Last_Rang
data_cleaned_bis["Last_Rang"] = data_cleaned_bis["Last_Rang"].astype(str)
data_cleaned_bis["Last_Rang"] = data_cleaned_bis["Last_Rang"].apply(lambda x : x.split("\n")[0])

# Ortho_Vitesse_kts
data_cleaned_bis["Ortho_Vitesse_kts"] = data_cleaned_bis["Ortho_Vitesse_kts"].astype(str)
data_cleaned_bis["Ortho_Vitesse_kts"] = data_cleaned_bis["Ortho_Vitesse_kts"].apply(lambda x : x.split(" ")[0])
data_cleaned_bis["Ortho_Vitesse_kts"] = data_cleaned_bis["Ortho_Vitesse_kts"][0:rank_RET-1].astype(float)

# Ortho_Distance_nm
data_cleaned_bis["Ortho_Distance_nm"] = data_cleaned_bis["Ortho_Distance_nm"].astype(str)
data_cleaned_bis["Ortho_Distance_nm"] = data_cleaned_bis["Ortho_Distance_nm"].apply(lambda x : x.split(" ")[0])
data_cleaned_bis["Ortho_Distance_nm"] = data_cleaned_bis["Ortho_Distance_nm"][0:rank_RET-1].astype(float)

# Rapport_Ortho_Fond_%
data_cleaned_bis["Rapport_Ortho_Fond_%"] = data_cleaned_bis["Rapport_Ortho_Fond_%"].astype(str)
data_cleaned_bis["Rapport_Ortho_Fond_%"] = data_cleaned_bis["Rapport_Ortho_Fond_%"].apply(lambda x : x.split(" ")[0])
data_cleaned_bis["Rapport_Ortho_Fond_%"] = data_cleaned_bis["Rapport_Ortho_Fond_%"][0:rank_RET-1].astype(float)

# Fond_Distance_nm
data_cleaned_bis["Fond_Vitesse_kts"] = data_cleaned_bis["Fond_Vitesse_kts"].astype(str)
data_cleaned_bis["Fond_Vitesse_kts"] = data_cleaned_bis["Fond_Vitesse_kts"].apply(lambda x : x.split(" ")[0])
data_cleaned_bis["Fond_Vitesse_kts"] = data_cleaned_bis["Fond_Vitesse_kts"][0:rank_RET-1].astype(float)

# Fond_Distance_nm
data_cleaned_bis["Fond_Distance_nm"] = data_cleaned_bis["Fond_Distance_nm"].astype(str)
data_cleaned_bis["Fond_Distance_nm"] = data_cleaned_bis["Fond_Distance_nm"].apply(lambda x : x.split(" ")[0])
data_cleaned_bis["Fond_Distance_nm"] = data_cleaned_bis["Fond_Distance_nm"][0:rank_RET-1].astype(float)

data_cleaned_bis

,Last_Rang,Nat_Voile,Skipper_Bateau,Date_Arrivee,Temps_Course_s,Ecart_Premier_s,Ecart_Precedent_s,Ortho_Vitesse_kts,Ortho_Distance_nm,Rapport_Ortho_Fond_%,Fond_Vitesse_kts,Fond_Distance_nm
1,1,FRA 17,Yannick Bestaven Maître Coq IV,28/01/2021 04:19:46 FR,80j 03h 44min 46s\n-10h 15min 00s,NaN,NaN,12.6,24365.7,117.3,14.8,28583.8
2,2,FRA 79,Charlie Dalin APIVIA,27/01/2021 20:35:47 FR,80j 06h 15min 47s\n,02h 31min 01s,02h 31min 01s,12.6,24365.7,119.6,15.1,29135.0
3,3,FRA 18,Louis Burton Bureau Vallée 2,28/01/2021 00:45:12 FR,80j 10h 25min 12s\n,06h 40min 26s,04h 09min 25s,12.6,24365.7,117.6,14.8,28650.0
4,4,FRA 01,Jean Le Cam Yes we Cam !,28/01/2021 20:19:55 FR,80j 13h 44min 55s\n-16h 15min 00s,10h 00min 09s,03h 19min 43s,12.5,24365.7,112.9,14.1,27501.5
5,5,MON 10,Boris Herrmann Seaexplorer - Yacht Club De Monaco,28/01/2021 11:19:45 FR,80j 14h 59min 45s\n-06h 00min 00s,11h 14min 59s,01h 14min 50s,12.6,24365.7,116.8,14.7,28448.5
6,6,FRA 59,Thomas Ruyant LinkedOut,28/01/2021 05:42:01 FR,80j 15h 22min 01s\n,11h 37min 15s,22min 16s,12.6,24365.7,119.7,15.1,29175.5
7,7,FRA 1000,Damien Seguin Groupe APICIL,28/01/2021 12:18:20 FR,80j 21h 58min 20s\n,18h 13min 34s,06h 36min 19s,12.5,24365.7,112.9,14.2,27512.3
8,8,ITA 34,Giancarlo Pedote Prysmian Group,28/01/2021 13:02:20 FR,80j 22h 42min 20s\n,18h 57min 34s,44min 00s,12.5,24365.7,116.9,14.7,28489.9
9,9,FRA 09,Benjamin Dutreux OMIA - Water Family,29/01/2021 10:05:20 FR,81j 19h 45min 20s\n,1j 16h 00min 34s,21h 03min 00s,12.4,24365.7,114.2,14.2,27832.5
10,10,FRA 53,Maxime Sorel V And B Mayenne,30/01/2021 04:50:15 FR,82j 14h 30min 15s\n,2j 10h 45min 29s,18h 44min 55s,12.3,24365.7,112.2,13.8,27346.9


In [11]:
# Temps_course
data_cleaned_bis["Temps_Course_s"] = data_cleaned_bis["Temps_Course_s"].astype(str)
data_cleaned_bis["Temps_Course_s"] = data_cleaned_bis["Temps_Course_s"][0:rank_RET-1].apply(lambda x : convert_race_time_seconds(x))

# Date_Arrivee
data_cleaned_bis["Date_Arrivee"] = data_cleaned_bis["Date_Arrivee"].astype(str)
data_cleaned_bis["Date_Arrivee"] = data_cleaned_bis["Date_Arrivee"][0:rank_RET-1].apply(lambda x : convert_date_arrivee(x))

# Ecart_Premier_s
data_cleaned_bis["Ecart_Premier_s"] = data_cleaned_bis["Ecart_Premier_s"].astype(str)
data_cleaned_bis["Ecart_Premier_s"] = data_cleaned_bis["Ecart_Premier_s"][1:rank_RET-1].apply(lambda x : convert_Ecart_time_seconds(x))


# Ecart_Premier_s
data_cleaned_bis["Ecart_Precedent_s"] = data_cleaned_bis["Ecart_Precedent_s"].astype(str)
data_cleaned_bis["Ecart_Precedent_s"] = data_cleaned_bis["Ecart_Precedent_s"][1:rank_RET-1].apply(lambda x : convert_Ecart_time_seconds(x))

data_cleaned_bis


,Last_Rang,Nat_Voile,Skipper_Bateau,Date_Arrivee,Temps_Course_s,Ecart_Premier_s,Ecart_Precedent_s,Ortho_Vitesse_kts,Ortho_Distance_nm,Rapport_Ortho_Fond_%,Fond_Vitesse_kts,Fond_Distance_nm
1,1,FRA 17,Yannick Bestaven Maître Coq IV,2021-01-28 04:19:46,6925486.0,NaN,NaN,12.6,24365.7,117.3,14.8,28583.8
2,2,FRA 79,Charlie Dalin APIVIA,2021-01-27 20:35:47,6934547.0,9061.0,9061.0,12.6,24365.7,119.6,15.1,29135.0
3,3,FRA 18,Louis Burton Bureau Vallée 2,2021-01-28 00:45:12,6949512.0,24026.0,14965.0,12.6,24365.7,117.6,14.8,28650.0
4,4,FRA 01,Jean Le Cam Yes we Cam !,2021-01-28 20:19:55,6961495.0,36009.0,11983.0,12.5,24365.7,112.9,14.1,27501.5
5,5,MON 10,Boris Herrmann Seaexplorer - Yacht Club De Monaco,2021-01-28 11:19:45,6965985.0,40499.0,4490.0,12.6,24365.7,116.8,14.7,28448.5
6,6,FRA 59,Thomas Ruyant LinkedOut,2021-01-28 05:42:01,6967321.0,41835.0,1336.0,12.6,24365.7,119.7,15.1,29175.5
7,7,FRA 1000,Damien Seguin Groupe APICIL,2021-01-28 12:18:20,6991100.0,65614.0,23779.0,12.5,24365.7,112.9,14.2,27512.3
8,8,ITA 34,Giancarlo Pedote Prysmian Group,2021-01-28 13:02:20,6993740.0,68254.0,2640.0,12.5,24365.7,116.9,14.7,28489.9
9,9,FRA 09,Benjamin Dutreux OMIA - Water Family,2021-01-29 10:05:20,7069520.0,144034.0,75780.0,12.4,24365.7,114.2,14.2,27832.5
10,10,FRA 53,Maxime Sorel V And B Mayenne,2021-01-30 04:50:15,7137015.0,211529.0,67495.0,12.3,24365.7,112.2,13.8,27346.9


In [12]:
# Traitement de la colonne "Nat_voile"
# On extrait le numéro de la voile et la nationnalité de la voile
data_cleaned_bis['Numero_voile'] = data_cleaned_bis['Nat_Voile'].apply(lambda x : extract_numbers(x))
data_cleaned_bis['Nat_Voile'] = data_cleaned_bis['Nat_Voile'].apply(lambda x : extract_characters(x))
    
# On constate encore des erreurs de format pour la colonne Nat_voile en effet, on remarque le caractère "FRFRA". 
# Cela est dû à un erreur de saisie des personnes en charge du site internet du Vendée Globle. 
#Il convient donc d'uniformiser le format des valeurs de la colonne.
# Pour ce faire, il convient de remplacer "FRFRA" => "FRA".
data_cleaned_bis['Nat_Voile'] = data_cleaned_bis['Nat_Voile'].apply(lambda x : x.replace("FRFRA", "FRA"))
    
# On crée un colonne contenant le nom du "Skipper" à partir de la colonne "Skipper_Bateau"
data_cleaned_bis['Skipper'] = data_cleaned_bis['Skipper_Bateau'].apply(lambda x : unidecode.unidecode((' '.join(x.split(' ')[0:2])).lower()))
    
# On crée une colonne qui contient la date du tableau excel (date d'enregistrement du classement)
i = "20210305_080000"
data_cleaned_bis['date_excel'] = "20210305_080000"
data_cleaned_bis['date_excel_datetime'] = datetime.strptime(i, '%Y%m%d_%H%M%S')

data_cleaned_bis

,Last_Rang,Nat_Voile,Skipper_Bateau,Date_Arrivee,Temps_Course_s,Ecart_Premier_s,Ecart_Precedent_s,Ortho_Vitesse_kts,Ortho_Distance_nm,Rapport_Ortho_Fond_%,Fond_Vitesse_kts,Fond_Distance_nm,Numero_voile,Skipper,date_excel,date_excel_datetime
1,1,FRA,Yannick Bestaven Maître Coq IV,2021-01-28 04:19:46,6925486.0,NaN,NaN,12.6,24365.7,117.3,14.8,28583.8,17,yannick bestaven,20210305_080000,2021-03-05 08:00:00
2,2,FRA,Charlie Dalin APIVIA,2021-01-27 20:35:47,6934547.0,9061.0,9061.0,12.6,24365.7,119.6,15.1,29135.0,79,charlie dalin,20210305_080000,2021-03-05 08:00:00
3,3,FRA,Louis Burton Bureau Vallée 2,2021-01-28 00:45:12,6949512.0,24026.0,14965.0,12.6,24365.7,117.6,14.8,28650.0,18,louis burton,20210305_080000,2021-03-05 08:00:00
4,4,FRA,Jean Le Cam Yes we Cam !,2021-01-28 20:19:55,6961495.0,36009.0,11983.0,12.5,24365.7,112.9,14.1,27501.5,01,jean le,20210305_080000,2021-03-05 08:00:00
5,5,MON,Boris Herrmann Seaexplorer - Yacht Club De Monaco,2021-01-28 11:19:45,6965985.0,40499.0,4490.0,12.6,24365.7,116.8,14.7,28448.5,10,boris herrmann,20210305_080000,2021-03-05 08:00:00
6,6,FRA,Thomas Ruyant LinkedOut,2021-01-28 05:42:01,6967321.0,41835.0,1336.0,12.6,24365.7,119.7,15.1,29175.5,59,thomas ruyant,20210305_080000,2021-03-05 08:00:00
7,7,FRA,Damien Seguin Groupe APICIL,2021-01-28 12:18:20,6991100.0,65614.0,23779.0,12.5,24365.7,112.9,14.2,27512.3,1000,damien seguin,20210305_080000,2021-03-05 08:00:00
8,8,ITA,Giancarlo Pedote Prysmian Group,2021-01-28 13:02:20,6993740.0,68254.0,2640.0,12.5,24365.7,116.9,14.7,28489.9,34,giancarlo pedote,20210305_080000,2021-03-05 08:00:00
9,9,FRA,Benjamin Dutreux OMIA - Water Family,2021-01-29 10:05:20,7069520.0,144034.0,75780.0,12.4,24365.7,114.2,14.2,27832.5,09,benjamin dutreux,20210305_080000,2021-03-05 08:00:00
10,10,FRA,Maxime Sorel V And B Mayenne,2021-01-30 04:50:15,7137015.0,211529.0,67495.0,12.3,24365.7,112.2,13.8,27346.9,53,maxime sorel,20210305_080000,2021-03-05 08:00:00


In [13]:
# On ré-organise l'ordre des colonnes du DataFrame
column_names = ["Last_Rang", "Numero_voile", "Nat_Voile",  "Skipper", "Skipper_Bateau", "Date_Arrivee", "Temps_Course_s", "Ecart_Premier_s", "Ecart_Precedent_s", "Ortho_Vitesse_kts", "Ortho_Distance_nm", "Rapport_Ortho_Fond_%", "Fond_Vitesse_kts", "Fond_Distance_nm", "date_excel", "date_excel_datetime"]
data_cleaned_bis = data_cleaned_bis.reindex(columns=column_names)

data_cleaned_bis

,Last_Rang,Numero_voile,Nat_Voile,Skipper,Skipper_Bateau,Date_Arrivee,Temps_Course_s,Ecart_Premier_s,Ecart_Precedent_s,Ortho_Vitesse_kts,Ortho_Distance_nm,Rapport_Ortho_Fond_%,Fond_Vitesse_kts,Fond_Distance_nm,date_excel,date_excel_datetime
1,1,17,FRA,yannick bestaven,Yannick Bestaven Maître Coq IV,2021-01-28 04:19:46,6925486.0,NaN,NaN,12.6,24365.7,117.3,14.8,28583.8,20210305_080000,2021-03-05 08:00:00
2,2,79,FRA,charlie dalin,Charlie Dalin APIVIA,2021-01-27 20:35:47,6934547.0,9061.0,9061.0,12.6,24365.7,119.6,15.1,29135.0,20210305_080000,2021-03-05 08:00:00
3,3,18,FRA,louis burton,Louis Burton Bureau Vallée 2,2021-01-28 00:45:12,6949512.0,24026.0,14965.0,12.6,24365.7,117.6,14.8,28650.0,20210305_080000,2021-03-05 08:00:00
4,4,01,FRA,jean le,Jean Le Cam Yes we Cam !,2021-01-28 20:19:55,6961495.0,36009.0,11983.0,12.5,24365.7,112.9,14.1,27501.5,20210305_080000,2021-03-05 08:00:00
5,5,10,MON,boris herrmann,Boris Herrmann Seaexplorer - Yacht Club De Monaco,2021-01-28 11:19:45,6965985.0,40499.0,4490.0,12.6,24365.7,116.8,14.7,28448.5,20210305_080000,2021-03-05 08:00:00
6,6,59,FRA,thomas ruyant,Thomas Ruyant LinkedOut,2021-01-28 05:42:01,6967321.0,41835.0,1336.0,12.6,24365.7,119.7,15.1,29175.5,20210305_080000,2021-03-05 08:00:00
7,7,1000,FRA,damien seguin,Damien Seguin Groupe APICIL,2021-01-28 12:18:20,6991100.0,65614.0,23779.0,12.5,24365.7,112.9,14.2,27512.3,20210305_080000,2021-03-05 08:00:00
8,8,34,ITA,giancarlo pedote,Giancarlo Pedote Prysmian Group,2021-01-28 13:02:20,6993740.0,68254.0,2640.0,12.5,24365.7,116.9,14.7,28489.9,20210305_080000,2021-03-05 08:00:00
9,9,09,FRA,benjamin dutreux,Benjamin Dutreux OMIA - Water Family,2021-01-29 10:05:20,7069520.0,144034.0,75780.0,12.4,24365.7,114.2,14.2,27832.5,20210305_080000,2021-03-05 08:00:00
10,10,53,FRA,maxime sorel,Maxime Sorel V And B Mayenne,2021-01-30 04:50:15,7137015.0,211529.0,67495.0,12.3,24365.7,112.2,13.8,27346.9,20210305_080000,2021-03-05 08:00:00


In [14]:
data_last_classement = data_cleaned_bis.copy()

#### On partage la variable _data_last_classement_ pour la réutiliser dans le notebook de l'analyse

In [15]:
data_last_classement_share = data_last_classement
%store data_last_classement_share

Stored 'data_last_classement_share' (DataFrame)
